<span style="font-size:larger;" font-family="Verdana">

### Introduction:

Boosting is the process of combining weak learners to generate a __Strong Rule__. A weak learner generate a rule (weak rule) from the training examples, this rule should be better than chance, this means the error should be less than 1/2.

* __Formal description of Boosting:__
<br><br>
   - Given training set  $(x_1,y_1),(x_2,y_2),...,(x_n,y_n)$, each instance  $x \in X$ we associate the correct label $y_i \in \{-1, 1\}$:
<br><br>
   - For t = 1,...,T :
        - Construct the distribution $D_t$ on $\{1,...,n\}$:
        - Find weak classifier: $h_t:$ $X \xrightarrow{} \{-1, 1\}$. The error $e_t$ on $D_t$ equal to $Pr_{i \sim D_t}[h_t(x_i)\ne y_i]$
<br><br>
   - Output the final Strong learner $H_\text{final}$


<span style="font-size:larger;" font-family="Verdana">
    
* __Example : AdaBoost__

    - The algorithm start by the uniforme distribution: $D_t(i)$ = $\dfrac{1}{n}$  
        - For t = 1,...,T : 
        - Given $D_t$, $h_t$ (set of weak learners).
        <br><br>
        - We update the distribution as follows:          
$$
D_{t+1}(i) = \dfrac{D_{t}(i)}{Z_t} \times \begin{cases} e^{-\alpha_t} &  y_i = h_t(x_i) \\e^{\alpha_t} &  y_i \ne h_t(x_i) \end{cases} = \dfrac{D_{t}(i)}{Z_t} \times \exp(-\alpha_t y_i h_t(x_i))    
$$  
          where $Z_t$ normalization factor, $\alpha_t$ = $\dfrac{1}{2} \ln(\dfrac{1 - e_t}{e_t})$
          <br><br>
    - The result classifier : $H(x) = sign(\sum_{t} \alpha_th_t(x))$ 


<span style="font-size:small;" font-family="Verdana">
    
**Notes**:  

   1. A week learner can be anything but the popular choice is _Boosting stumps_ ( one-level decision trees ). 
   <br><br>
   2. Adaboost can be viewd as gradient descent over an exponential loss function :  
- _Prediction_: $H_t(x) = \sum_{t} \alpha_th_t(x)$.  
- _Loss_: $L(F_t(x),y)$ Total loss is: $\sum_{i=1}^{n} L(F_t(x_i),y_i) )$  




### Application: XGBoost  


XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. The same code runs on major distributed environment (Hadoop, SGE, MPI) and can solve problems beyond billions of examples. 


[Xgboost documentation](https://xgboost.readthedocs.io/en/latest/)

#### Types whales classification:

This example is based on underwater sound data collected in the Gulf of Mexico during and since the time of the Deepwater Horizon oil spill in 2010. The task is to assess what species of whales were present at three recording sites,

The classification was performed using the echo-location clicks emitted by beaked whales. These are very short pulses (less than a ms) that whales and dolphins use to sense their surroundings, including the detection of their prey. Different species of beaked whales emit clicks with different waveforms and spectral distributions. The time intervals between clicks also provides information about the species. Examples clicks from Cuvier’s and Gervais’ beaked whales are shown below.


<img src="./img.jpg" height=500 width=500>  


This notebook don't include data processing  

_The goal of the project is to create a classifier that takes as input data a run of clicks and outputs the species._

### Imports

In [1]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pickle
import random
%matplotlib inline

### Data

In [2]:
with open('Data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('Data/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open('Data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('Data/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

### Setting Parameters for XG Boost  

* Maximum Depth of the Tree = 3 _(maximum depth of each decision trees)_
* Step size shrinkage used in update to prevents overfitting = 0.3 _(how to weigh trees in subsequent iterations)_
* Evaluation Criterion= Maximize Loglikelihood according to the logistic regression _(logitboost)_
* Maximum Number of Iterations = 1000 _(total number trees for boosting)_
* Early Stop if score on Validation does not improve for 5 iterations


In [3]:
def xgboost_plst():
    param = {}
    param['max_depth']= 3   # depth of tree
    param['eta'] = 0.3      # shrinkage parameter
    param['silent'] = 1     # not silent
    param['objective'] = 'binary:logistic'
    param['nthread'] = 7 # Number of threads used
    param['eval_metric'] = 'logloss'
    plst = param.items()
    return plst

### Helper Functions

* The function <font color="blue">calc_stats</font> takes the xgboost margin scores as input and returns two numpy arrays   min_scr, max_scr which are calculated as follows:

    1. **min_scr**: mean - (3 $\times$ std)
    2. **max_scr**: mean + (3 $\times$ std)

    Here the input margin scores, represents the processed XGBoost margin scores obtained from the <font color="blue">bootstrap_pred</font> function. Each row represents the various scores for a specific example in an iteration and your <font color="blue">calc_stats</font> function is supposed to compute the **min_scr** and **max_scr** as defined for each example. 


* The function <font color="blue">predict</font> takes the minimum score array and maximum score array and returns predictions as follows:

    1. If respective minimum score and maximum score values are less than 0, predict -1 (**Cuvier's**)
    2. If respective minimum score value is less than 0 and maximum score value is greater than 0, predict 0 (**Unsure**)
    3. If respective minimum score and maximum score values are greater than 0, predict 1 (**Gervais**)

    Based on the ranges for each of the examples, i.e, (min_scr, max_scr), we can predict whether it's a Gervais or a Cuvier. Since we take margin scores from a set of bootstraps for each example, we use the minimum and maximum score arrays to predict to determine the classification.

In [4]:
def calc_stats(margin_scores):
    arr_min_scr = []
    arr_max_scr = []
    
    for arr in margin_scores:
        m = arr.mean()
        sd = arr.std()
        arr_min_scr.append(round(m-3*sd, 2))
        arr_max_scr.append(round(m+3*sd, 2))
    
    return (np.asarray(arr_min_scr), np.asarray(arr_max_scr))


def predict(min_scr, max_scr):
    
    answer = []
    
    for i in range(len(min_scr)):
        if(min_scr[i] < 0 and max_scr[i] < 0):
            answer.append(-1)
        if(min_scr[i] < 0 and max_scr[i] > 0):
            answer.append(0)
        if(min_scr[i] > 0 and max_scr[i] > 0):
            answer.append(1)
            
    return np.asarray(answer)

### Calculating margins scores

#### Main API

* `xgboost.train` is the learning API that trains the Gradient Boosting Model,
   * The main parameters are:
      * **plst** – XGBoost parameter list
      * **dtrain** – Data to be trained
      * **num_round** – Number of iterations of boosting. (default: 100)
      * **evallist** – List of items to be evaluated during training
      * **verbose_eval** - This can be used to control how much information the train function prints. You might want to set to **False** to avoid printing logs.
* `bst.predict` is the API that makes score predictions
   * The main parameters are:
      * **dtest** – Test Data
      * **dtrain** – Data to be trained
      * **ntree_limit** – Limit number of trees in the prediction (Use: ntree_limit=bst.best_ntree_limit)
      * **output_margin** - Whether to output the raw untransformed margin value (Use: output_margin=True)


#### Procedure

Repeat the given procedure for n_bootstrap number of iterations:

For **n_bootstrap** iterations:
* Sample **boostrap_size** indices from the training set **with replacmennt**
* Create train and test data matrices (dtrain, dtest) using xgb.DMatrix(X_sample, label=y_sample)
* Initialise the evallist parameter [(dtrain, 'train'), (dtest, 'eval')]
* Train the model using the XGBoost train API and make score predictions using bst.predict object returned by XGB train API. Ensure you set **output_margin=True** to get raw untransformed output scores.
* Normalize them by dividing them with the normalizing factor as max(scores) - min(scores) and round these values to a precision of two decimal places

Then: 
* For each individual example, remove scores below the minRth percentile and greater than maxRth percentile (sort for each example if necessary)
* Call the calc_stats function to compute min_scr and max_scr with the filtered margin scores as parameter
* Return the min_scr and max_scr computed by the **calc_stat** function using the margin scores.
      

In [5]:
def bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap, minR,maxR, bootstrap_size, num_round=100, plst=xgboost_plst()):
    
    answer = []
    
    for i in range(n_bootstrap):
        rdc =  np.random.choice(bootstrap_size, bootstrap_size)
        xsample = X_train[rdc]
        ysample = y_train[rdc]
        
        dtrain = xgb.DMatrix(xsample, label=ysample)
        dtest = xgb.DMatrix(X_test, label=y_test)
        evallist =  [(dtrain, 'train'), (dtest, 'eval')]
        
        bst = xgb.train(plst, dtrain, num_boost_round=num_round, evals=evallist, early_stopping_rounds=5, verbose_eval=False)
        margins_scores = bst.predict(dtest, output_margin=True, ntree_limit=bst.best_ntree_limit, validate_features=True)
        
        MX = margins_scores.max()
        MI = margins_scores.min()
        margins_scores = [round(x/(float(MX-MI)),2) for x in margins_scores]
        
        margins_scores = np.asarray(margins_scores)
        sp = margins_scores.shape[0]
        margins_scores = margins_scores.reshape(sp,-1)
        
        if(len(answer) == 0):
            answer = np.ones((sp,1)) + maxR
        
        answer = np.concatenate((answer,margins_scores), axis=1)
        
        for i in range(len(answer)):
            answer[i] = answer[i][(answer[i] < maxR)]
        
        
    return calc_stats(answer)

In [6]:
def process(X_train, X_test, y_train, y_test, n_bootstrap=100):
    min_scr, max_scr = bootstrap_pred(X_train, X_test, y_train, y_test, n_bootstrap=n_bootstrap, \
                                            minR=0.1, maxR=0.9, bootstrap_size=len(X_train))
    pred = predict(min_scr, max_scr)
    return min_scr, max_scr, pred

### Testing

In [8]:
sample_indices = np.load('Data/vis_indices.npy')
X_test_samp = X_test[sample_indices]
y_test_samp = np.array(y_test[sample_indices], dtype=int)
midpt = np.load('Data/vis_midpt.npy')
avg_length = np.load('Data/vis_avg_length.npy')
min_scr, max_scr, predictions = process(X_train, X_test_samp, y_train, y_test_samp)
length = max_scr - min_scr

In [9]:
predictions

array([ 0, -1, -1, -1,  0,  1,  1, -1,  1,  0])

In [10]:
min_scr

array([-0.24, -0.63, -0.65, -0.63, -0.18,  0.31,  0.24, -0.57,  0.42,
       -0.29])

In [11]:
max_scr

array([ 0.33, -0.31, -0.12, -0.31,  0.37,  0.61,  0.62, -0.47,  0.54,
        0.47])